<a href="https://colab.research.google.com/github/vjhawar12/FreshNET/blob/main/FreshNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=2hDyy1VYRxMwp31IG3oT9g4QqA3lzh&prompt=consent&token_usage=remote&access_type=offline&code_challenge=9qVHTsGss38khnJHTxlMRpz1Fa5dPabujUwwlp7ZfqE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 

In [ ]:
!gcloud config set project freshnet-466505

In [ ]:
!cd /content/dataset && gcloud storage cp --recursive gs://fruit-images-freshnet .

In [ ]:
!cd /content/dataset && ls

[Errno 2] No such file or directory: '/content/dataset && ls'
/content


In [ ]:
path_to_train_imgs = ""
path_to_test_imgs = ""

BATCH_SIZE = 256
RANDOM_SEED = 42
TRAIN_SIZE, VAL_SIZE = 0.8, 0.2
IMG_SIZE = 320 # Original images are ~400*400 px so resizing them to 320 retains detail while reducing computational cost

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE, IMG_SIZE),
    # additional preprocessing
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize(IMG_SIZE, IMG_SIZE),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ToTensor(),
])

In [ ]:
full_train_data = ImageFolder(path_to_train_imgs, transform=train_transform)
test_data = ImageFolder(path_to_test_imgs, transform=test_transform)
train_data, val_data = random_split(full_train_data, [TRAIN_SIZE, VAL_SIZE], generator=RANDOM_SEED)

print(f"{full_train_data.class_to_idx} \n {train_data.class_to_idx} \n {test_data.class_to_idx}")

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=10)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
class InvertedResidual(nn.Module):
  def __init__(self, in_channels, out_channels, exp_factor, downsample_factor, kernel_size=3):
    super().__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.exp_factor = exp_factor
    self.downsample_factor = downsample_factor
    self.kernel_size = 3

    out = self.in_channels * self.exp_factor

    self.block = nn.Sequential(
        nn.Conv2d(in_channels=self.in_channels, out_channels=out, kernel_size=1, stride=1, groups=1), # expansion
        nn.Conv2d(in_channels=out, out_channels=out, kernel_size=self.kernel_size, stride=self.downsample_factor, groups=out), # depthwise
        nn.Conv2d(in_channels=out, out_channels=self.out_channels, kernel_size=1, stride=1, groups=1), # pointwise
    )

  def forward(self, x):
    x = self.block(x)
    return x

In [ ]:
class FreshNET_CNN(nn.Module):
  """
  FreshNET tries to apply the concept of pointwise and depthwise convolutions outlined in the MobileNET original papers. The idea is instead of sliding a N*N*3 filter
  across the input image, which does channel mixing and spatial filtering together, we can do them independently. This is supposed to save computation and be more efficient.
  """
  def __init__(self):
    super().__init__()
    initial_conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2) # 320*320*3 --> 160*160*32

    # inverted residual block 1: 160*160*32 --> 160*160*16
    block_1 = nn.Sequential(
        InvertedResidual(32, 16, 1, 1)
    )

    # inverted residual block 2: 160*160*16 --> 80*80*24, exp=6
    block_2 = nn.Sequential(
        InvertedResidual(16, 24, 6, 2),
        InvertedResidual(24, 24, 6, 1)
    )

    # inverted residual block 3: 80*80*24 --> 40*40*32, exp=6
    block_3 = nn.Sequential(
        InvertedResidual(24, 32, 6, 2),
        InvertedResidual(32, 32, 6, 1),
        InvertedResidual(32, 32, 6, 1),
    )

    # inverted residual block 4: 40*40*32 --> 20*20*64, exp=6
    block_4 = nn.Sequential(
        InvertedResidual(32, 64, 6, 2),
        InvertedResidual(64, 64, 6, 1),
        InvertedResidual(64, 64, 6, 1),
        InvertedResidual(64, 64, 6, 1),
    )

    # inverted residual block 5: 20*20*64 --> 20*20*96, exp=6
    block_5 = nn.Sequential(
        InvertedResidual(64, 96, 6, 1),
        InvertedResidual(96, 96, 6, 1),
        InvertedResidual(96, 96, 6, 1),
    )

    # inverted residual block 6: 20*20*96 --> 10*10*160, exp=6
    block_5 = nn.Sequential(
        InvertedResidual(96, 160, 6, 2),
        InvertedResidual(160, 160, 6, 1),
        InvertedResidual(160, 160, 6, 1),
    )

    # inverted residual block 7: 10*10*160 --> 10*10*320
    block_6 = nn.Sequential(
        InvertedResidual(160, 320, 6, 1)
    )

    final_conv = nn.Conv2d(in_channels=320, out_channels=1280, kernel_size=1, stride=1) # 10*10*320 --> 10*10*1280

    max_pool = nn.MaxPool2d(10) # 10*10*1280 --> 1*1*1280

    fc = nn.Linear(in_features=1280, out_features=1000)

    self.layers = nn.ModuleList(
        [initial_conv, block_1, block_2, block_3, block_4, block_5, block_6, final_conv, max_pool, fc]
    )

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)

    return x

In [ ]:
cnn = FreshNET_CNN()

In [ ]:
def train():
  pass

In [ ]:
def validate():
  pass

In [ ]:
def test():
  pass